In [ ]:
import math
import pandas as pd
import scipy.constants as const
import scipy.integrate as integrate
from scipy.interpolate import interp1d
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

# Constants
h = const.h  # Planck constant
c = const.c  # Speed of light
k = const.k  # Boltzmann constant
T = 5778  # Temperature of the Sun's surface in Kelvin

lam_min = 300e-9  # Minimum wavelength in meters
lam_max = 2500e-9  # Maximum wavelength in meters


In [ ]:

# Energy-related calculations
file_path = 'OpticalPropertiesOfSilicon.xlsx'  # Adjust the file path as needed
df = pd.read_excel(file_path)

# Extract wavelengths and absorption coefficients into arrays
# Assuming the Excel file has columns 'Wavelength' (in nanometers) and 'AbsorptionCoefficient' (in cm^-1)
wavelengths_nm = df['wavelength(nm)'].to_numpy()  # Wavelengths in nanometers
absorption_coefficients_cm = df['a(/cm)'].to_numpy()  # Absorption coefficients in cm^-1

# Convert wavelengths to meters and absorption coefficients to m^-1
wavelengths_m = wavelengths_nm * 1e-9  # Convert nanometers to meters
absorption_coefficients_m = absorption_coefficients_cm * 100  # Convert cm^-1 to m^-1

# Create an interpolation function for absorption coefficients
absorption_interpolator = interp1d(wavelengths_m, absorption_coefficients_m, kind='cubic', fill_value="extrapolate")

def silicon_absorption_coefficient(wavelength):
    """
    Calculate the absorption coefficient of silicon for a given wavelength using interpolation.
    wavelength: Wavelength of the incoming light (in meters)
    """
    return absorption_interpolator(wavelength)


# Update the photon calculation to include wavelength-dependent absorption
def calculate_photon_absorption(intensity):
    """
    Calculate the number of photons absorbed by silicon over the solar spectrum.
    intensity: Intensity of sunlight (W/m²)
    lam_min, lam_max: Wavelength range (in meters)
    """
    def integrand(lam):
        energy_per_photon = h * c / lam
        num_photons = intensity / energy_per_photon
        absorption_efficiency = silicon_absorption_coefficient(lam)
        return num_photons * absorption_efficiency

    absorbed_photons, _ = integrate.quad(integrand, lam_min, lam_max)
    return absorbed_photons


df_results = pd.read_csv('output.csv')
df_results['Absorbed Photons'] = df_results['Sunlight Intensity Post Intake'].apply(calculate_photon_absorption)
df_results['exciton_generation_rate'] = df_results['Absorbed Photons'] # Assuming one exciton per absorbed photon
df_results.to_csv('output.csv')

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))  # Adjust the figure size if needed
plt.plot(df_results.index, df_results['exciton_generation_rate'], marker='o', linestyle='-')

# Set labels and title
plt.xlabel('Hour')
plt.ylabel('Exciton generation rate')
plt.title('Exciton generation rate vs Hour')

plt.savefig('exciton_generation_rate.png')
plt.show()